**RUN rpy2: a library to run R in Python**

In [1]:
 %%capture
 %load_ext rpy2.ipython
 !pip install --pyarrow  

In [2]:
%%capture
import pandas as pd
import numpy as np
from glob import glob
import pyarrow.feather as ft
from statsmodels.tsa.stattools import adfuller
import os

In [3]:
%%R
install.packages("lubridate")
install.packages('tseries')
install.packages('xts')
install.packages('feather',LIBARROW_MINIMAL=false)
install.packages('pracma')
install.packages('stats')
install.packages('ggplot2')


# devtools::install_github("wesm/feather/R")

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/lubridate_1.8.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 475704 bytes (464 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [4]:
files_cpi=glob('/content/drive/MyDrive/Library/data_amar/cpi_inflation_amar/*.xlsx')
files_sna=glob('/content/drive/MyDrive/Library/data_amar/national_accounts_amar/*.xlsx')
files_labor=glob('/content/drive/MyDrive/Library/data_amar/labor_population_amar/*.xlsx')

**Create a monthly date sequence that can be used in pythonn**

In [5]:
%%capture
%%R -o date -o qdate                    # Install & load lubridate
library("lubridate")
date <- seq(as.Date('1982/3/21'),as.Date('2021/8/25'),by='month')
qdate <- paste0(year(date), "-0",quarter(date))
date=format(date,'%Y-%m')

RInterpreterError: ignored

**Create dataframe of monthly and quarterly cpi**

In [6]:
file_1=[file for file in files_cpi if '6_base_95_no_detail' in file]
cpi=pd.read_excel(file_1[0])
cpi.columns=['year','month','cpi']
cpi=cpi.iloc[1:475,].reset_index(drop=True)
year_=np.arange(1361,1401)
year=np.repeat(year_,12)
month=np.tile(np.arange(1,13),40)
df=pd.DataFrame({'year':year,'month':month})
df['month']=df['month'].apply(lambda x: str(x).zfill(2))
df['jdate']=df['year'].astype(str)+'-'+df['month'].astype(str)
df.drop(df.tail(6).index,inplace=True)
cpi['date']=date
cpi['jdate']=df['jdate']
cpi.drop(['month','year'],inplace=True,axis=1)
cpi['date']=pd.to_datetime(cpi['date'])
cpi=cpi[cpi.columns[::-1]]


#create quarterly cpi


cpi['cpi']=pd.to_numeric(cpi.cpi)
pd.set_option('display.max_rows',160)
cpi[['jyear','jmonth']]=cpi['jdate'].str.split('-',expand=True)
cpi['jmonth']=cpi['jmonth'].astype(int).apply(lambda x: np.where(x<=3,1,x))
cpi['jmonth']=cpi['jmonth'].astype(int).apply(lambda x: np.where(3<x<=6,2,x))
cpi['jmonth']=cpi['jmonth'].astype(int).apply(lambda x: np.where(6<x<=9,3,x))
cpi['jmonth']=cpi['jmonth'].astype(int).apply(lambda x: np.where(9<x<=12,4,x))
cpi['jmonth']=cpi['jmonth'].astype(str).str.zfill(2)
cpi.rename(columns={'jmonth':'jquarter'},inplace=True)
cpi_quarterly=cpi.groupby(['jyear','jquarter']).mean().reset_index()
cpi_quarterly['jdate']=cpi_quarterly['jyear'].astype(str)+"-"+cpi_quarterly['jquarter'].astype(str)
cpi_quarterly=cpi_quarterly.iloc[:,2:4]
cpi_quarterly['date']=pd.Series(list(dict.fromkeys(qdate)))
cpi_quarterly=cpi_quarterly[cpi_quarterly.columns[::-1]]

NameError: ignored

**Create dataframe of quarterly gdp**

In [ ]:
gdp_ts=pd.read_excel(r'/content/drive/MyDrive/Library/data - kalancbi/sna/gdp_ts.xlsx')
gdp_ts[['year','month']]=gdp_ts['quarter'].str.split('-',1,expand=True)
gdp_ts['month']=gdp_ts['month'].str.zfill(2)
gdp_ts['jdate']=gdp_ts['year'].astype(str)+'-'+gdp_ts['month'].astype(str)
# gdp_ts=gdp_ts.set_index('jdate')

**Merge quarterly cpi with quarterly gdp**

In [ ]:
df=cpi_quarterly.merge(gdp_ts[['current_83_90_95','rebased_83_to_95','jdate']],how='outer',on='jdate')
df.rename(columns={'current_83_90_95':'nominal_gdp', 'rebased_83_to_95':'real_gdp'},inplace=True)
cpi_lag=df['cpi'].shift(1)
df['inflation']=(df['cpi']/cpi_lag)-1
os.chdir(r'/content/drive/MyDrive')
ft.write_feather(df,'\content\drive\MyDrive\cpi_gdp_df.feather')
df.to_csv('cpi_gdp.csv',index=False)

**TEDPIX 1388-1400**

In [ ]:
path='/content/drive/MyDrive/Library/data_bourse/TEDPIX_archive'
files_index=glob(path+'/*.xlsx')
TEDPIX=pd.DataFrame()
for file in files_index:
  df=pd.read_excel(file)
  TEDPIX=TEDPIX.append(df)
TEDPIX.columns=['jdate','index_name','index']
TEDPIX.reset_index(inplace=True)

In [ ]:
tedpix_lag=TEDPIX['index'].shift(-1)
tedpix_daily_ret=(tedpix_lag/TEDPIX['index'])-1
tedpix_daily_ret=tedpix_daily_ret.reset_index(drop=True)
tedpix_daily_ret.plot()
print(tedpix_daily_ret[2999:3092].sum())
print(TEDPIX['index'][3092],TEDPIX['index'][3000],(TEDPIX['index'][3092]/TEDPIX['index'][3000])-1)

In [ ]:
float_index['index'][3092]/float_index['index'][3091]
tedpix_daily_ret[3091]

**df in R (via csv file)**

**set date column as.Date**

**change df to time series**

In [ ]:
%%R
df <- read.csv('cpi_gdp.csv')
df$date <- seq(as.Date('1982/3/1'),as.Date('2021/8/1'),'quarter')
df_ts <- xts(df[,-1],order.by=as.Date(df[,1]))

**plot time series**

In [ ]:
index_lag=float_index['index'].shift(1)
float_index_ret=((float_index['index']-index_lag)/index_lag)
float_index_ret.iloc[2000:2900,].plot(figsize=(12,10))

In [ ]:
%%R
plot(x=df$date,y=df$cpi,'b')
plot(x=df$date,y=df$real_gdp,'b')
plot(x=df$date,y=df$nominal_gdp,'b')
# plot(df&)

In [ ]:
%%R
inflation_df<-df[c('inflation','date')][-1,]
plot(x=inflation_df$date,y=inflation_df$inflation,'l')

**Augmented Dickey-Fuller (ADF) t-statistics test for unit root:** *`level variables`*

In [ ]:
%%R
adf.test(df_ts$cpi)

In [ ]:
%%R
adf.test(df_ts[!is.na(df_ts$real_gdp),]$real_gdp)

In [ ]:
%%R
adf.test(df_ts[!is.na(df_ts$nominal_gdp),]$nominal_gdp)

**Augmented Dickey-Fuller (ADF) t-statistics test for unit root:** *`log variables`*



In [ ]:
%%R
adf.test(log(df$cpi))

In [ ]:
%%R
adf.test(log(df[!is.na(df$real_gdp),]$real_gdp))

In [ ]:
%%R
adf.test(log(df[!is.na(df$nominal_gdp),]$nominal_gdp))

**Augmented Dickey-Fuller (ADF) t-statistics test for unit root:** *`delta_log variables`*

**warning**: "*`There is an issue of whether the variables in a VAR need to be stationary. Sims
(1980) and Sims, Stock, and Watson (1990) recommended against differencing even
if the variables contain a unit root. They argued that the goal of a VAR analysis is to
determine the interrelationships among the variables, not to determine the parameter
estimates. The main argument against differencing is that it “throws away” information
concerning the comovements in the data (such as the possibility of cointegrating
relationships). Similarly, it is argued that the data need not be detrended`*"

Enders, W. (2014) Applied Econometric Time Series. 4th Edition. John Wiley, New York.



In [ ]:
%%R
adf.test(diff(log(df$cpi)))

In [ ]:
%%R
adf.test(diff(log(df[!is.na(df$real_gdp),]$real_gdp)))

In [ ]:
%%R
adf.test(diff(log(df[!is.na(df$nominal_gdp),]$nominal_gdp)))

**detrend function**

compute residuals of cpi, real_gdp and nominal_gdp

In [ ]:
%%R
library('pracma')
cpi_res<-detrend(df$cpi,'linear')
real_gdp_res<-detrend(df$real_gdp,'linear')
nominal_gdp_res<-detrend(df$nominal_gdp,'linear')

In [ ]:
%%R
plot(cpi_res)
plot(real_gdp_res)
plot(nominal_gdp_res)

**LINEAR REGRESSION MODEL**

In [ ]:
%%R
library('stats')
real_gdp<-df[-c(0:24),]
real_gdp<-real_gdp[-c(133,134),]
real_gdp<-real_gdp[c('date','real_gdp')]
real_gdp_lm<-lm(real_gdp$real_gdp~real_gdp$date)
coef(real_gdp_lm)

In [ ]:
%%R
qqnorm(resid(real_gdp_lm))

In [ ]:
%%R
acf(resid(real_gdp_lm))

In [ ]:
%%R
pacf(resid(real_gdp_lm))